In [ ]:
import json
import requests

# -------------------------------
# 1. CONFIGURATION DE L’ENDPOINT
# -------------------------------
endpoint_url = "https://stock-anomaly-endpoint.eastus.inference.ml.azure.com/score"   # <--- à remplacer
api_key = "PUT_YOUR_KEY_HERE_AFTER_PUSH"              # <--- à remplacer

# IMPORTANT : le nom exact de ton déploiement
deployment_name = "stock-anomaly-detection-model-2"

# -------------------------------
# 2. DONNÉES D’ENTRÉE À TESTER
# -------------------------------
payload = {
    "input_data": {
        "data": [
            [125.3, 127.1, 124.8, 126.6, 3400000]  # Exemple
        ]
    }
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
    "azureml-model-deployment": deployment_name   # <<< CLÉ MAGIQUE
}

# -------------------------------
# 3. ENVOI DE LA REQUÊTE
# -------------------------------
response = requests.post(
    endpoint_url,
    headers=headers,
    data=json.dumps(payload)
)

print("Status code:", response.status_code)
print("Résultat :")
print(response.json())


Status code: 200
Résultat :
{'anomaly_predictions': [-1], 'anomaly_scores': [-0.17093744879636041], 'is_anomaly': [1], 'message': "Détection d'anomalies réussie", 'anomalies_count': 1}


In [ ]:
%pip install yfinance


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     ---------------------------------- ----- 2.6/3.0 MB 18.9 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 17.3 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15617 sha256=db92b4253420d8066140af461e9df88b132394725be5fe4ec82ac7882ef75e8c
  Stored in directory: c:\user

In [3]:
import yfinance as yf
import pandas as pd

# Ex : l'action Microsoft (MSFT) sur les dernières minutes / heures
ticker = "MSFT"
data = yf.download(ticker, period="5d", interval="1h")  # 5 jours, bougies 1h

data.tail()


C:\Users\HH\AppData\Local\Temp\ipykernel_23648\3398490792.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="5d", interval="1h")  # 5 jours, bougies 1h
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,MSFT,MSFT,MSFT,MSFT,MSFT
Datetime,,,,,
2025-11-18 16:30:00+00:00,488.849915,494.440002,488.630005,492.204987,2283365
2025-11-18 17:30:00+00:00,492.130005,493.329987,487.899994,488.804993,6580288
2025-11-18 18:30:00+00:00,493.079987,493.470001,491.605011,492.160004,1125959
2025-11-18 19:30:00+00:00,495.260010,495.269989,492.810089,493.130005,1562396
2025-11-18 20:30:00+00:00,493.799988,495.739899,493.500000,495.260010,2226610


In [4]:
# On prend la dernière ligne pour simuler du temps réel
last_row = data.tail(1)

features = last_row[['Open', 'High', 'Low', 'Close', 'Volume']].values.tolist()
features


[[495.260009765625, 495.7398986816406, 493.5, 493.79998779296875, 2226610.0]]

In [ ]:
import json
import requests

endpoint_url = "https://stock-anomaly-endpoint.eastus.inference.ml.azure.com/score"   # REST endpoint
api_key = "PUT_YOUR_KEY_HERE_AFTER_PUSH"              # Primary key
deployment_name = "stock-anomaly-detection-model-2"

payload = {
    "input_data": {
        "data": features   # les valeurs extraites de Yahoo Finance
    }
}

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
    "azureml-model-deployment": deployment_name
}

response = requests.post(endpoint_url,
                         headers=headers,
                         data=json.dumps(payload))

print("Status code:", response.status_code)
print("Réponse :", response.json())


In [ ]:
import yfinance as yf
import pandas as pd
import time
import json
import requests
from datetime import datetime


In [ ]:
# === CONFIG AZURE ML ENDPOINT ===

endpoint_url = "https://stock-anomaly-endpoint.eastus.inference.ml.azure.com/score"      # <-- colle ici l'URL REST
api_key = "PUT_YOUR_KEY_HERE_AFTER_PUSH"                 # <-- colle ici la Primary Key
deployment_name = "stock-anomaly-detection-model-2"  # nom du déploiement

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}",
    "azureml-model-deployment": deployment_name
}


In [ ]:
def get_latest_stock_point(ticker="MSFT"):
    # On récupère les dernières minutes/heures de données
    df = yf.download(ticker, period="1d", interval="5m")  # 1 jour, bougie 5 minutes

    if df.empty:
        return None, None

    last_row = df.tail(1)
    
    # Features utilisées pour l’IsolationForest (adapter si besoin)
    features = last_row[['Open', 'High', 'Low', 'Close', 'Volume']].values.tolist()[0]
    
    # Time index (timestamp)
    ts = last_row.index[0]
    return ts, features


In [ ]:
def call_anomaly_detector(features):
    payload = {
        "input_data": {
            "data": [features]   # liste de listes
        }
    }
    response = requests.post(
        endpoint_url,
        headers=headers,
        data=json.dumps(payload)
    )
    if response.status_code != 200:
        print("Erreur:", response.status_code, response.text)
        return None
    
    return response.json()


In [ ]:
results = []

ticker = "MSFT"     # tu peux changer: AAPL, TSLA, etc.
interval_seconds = "20s"  # une requête chaque minute
num_iterations = 10    # par exemple : 10 itérations pour tester

for i in range(num_iterations):
    print(f"\n--- Itération {i+1}/{num_iterations} ---")
    ts, features = get_latest_stock_point(ticker)
    
    if ts is None:
        print("Pas de données récupérées.")
        time.sleep(interval_seconds)
        continue

    print("Timestamp:", ts)
    print("Features:", features)
    
    resp = call_anomaly_detector(features)
    
    if resp is None:
        print("Erreur lors de l'appel au modèle.")
        time.sleep(interval_seconds)
        continue

    is_anomaly = resp.get("is_anomaly", [0])[0]
    score = resp.get("anomaly_scores", [None])[0]
    
    print("is_anomaly:", is_anomaly, "score:", score)

    results.append({
        "timestamp": ts,
        "ticker": ticker,
        "open": features[0],
        "high": features[1],
        "low": features[2],
        "close": features[3],
        "volume": features[4],
        "is_anomaly": is_anomaly,
        "score": score
    })
    
    # Attente avant la prochaine itération
    time.sleep(interval_seconds)

df_results = pd.DataFrame(results)
df_results
